In [14]:
import torch

In [15]:
with open('input.txt', 'r', encoding='utf8') as f:
    text = f.read()

In [16]:
chars = sorted(list(set(text)))
vocab_size = len(chars)

itos = { i:s for i,s in enumerate(chars)}
stoi = { s:i for i,s in enumerate(chars)}

encode = lambda s: [stoi[c] for c in s]
decode = lambda l: ''.join([itos[i] for i in l])

# print(encode('hello'))
# print(decode([46, 43, 50, 50, 53]))

In [19]:
data = torch.tensor(encode(text), dtype=torch.long)
print(data.shape)
print(data[:100])

torch.Size([1115394])
tensor([18, 47, 56, 57, 58,  1, 15, 47, 58, 47, 64, 43, 52, 10,  0, 14, 43, 44,
        53, 56, 43,  1, 61, 43,  1, 54, 56, 53, 41, 43, 43, 42,  1, 39, 52, 63,
         1, 44, 59, 56, 58, 46, 43, 56,  6,  1, 46, 43, 39, 56,  1, 51, 43,  1,
        57, 54, 43, 39, 49,  8,  0,  0, 13, 50, 50, 10,  0, 31, 54, 43, 39, 49,
         6,  1, 57, 54, 43, 39, 49,  8,  0,  0, 18, 47, 56, 57, 58,  1, 15, 47,
        58, 47, 64, 43, 52, 10,  0, 37, 53, 59])


In [28]:
n = int(0.9*len(data))
train_data = data[:n]
val_data = data[n:]
print(train_data.shape, val_data.shape)


torch.Size([1003854]) torch.Size([111540])


tensor([18, 47, 56, 57, 58])

In [32]:
torch.manual_seed(1337)

batch_size = 4
block_size = 8

def get_batch(split):
    data = train_data if split == 'train' else val_data
    ix = torch.randint(len(data) - block_size, (batch_size, ))
    x = torch.stack([data[i:i+block_size] for i in ix])
    y = torch.stack([data[i+1:i+block_size+1] for i in ix])
    return x,y

xb, yb = get_batch('train')
print('inputs:')
print(xb.shape)
print(xb)
print('targets:')
print(yb.shape)
print(yb)

print('----')

for b in range(batch_size):
    for t in range(block_size):
        context = xb[b, :t+1]
        target = yb[b, t]
        print(f"when input is {context.tolist()} the target is: {target}")

[tensor([24, 43, 58,  5, 57,  1, 46, 43]), tensor([44, 53, 56,  1, 58, 46, 39, 58]), tensor([52, 58,  1, 58, 46, 39, 58,  1]), tensor([25, 17, 27, 10,  0, 21,  1, 54])]
inputs:
torch.Size([4, 8])
tensor([[24, 43, 58,  5, 57,  1, 46, 43],
        [44, 53, 56,  1, 58, 46, 39, 58],
        [52, 58,  1, 58, 46, 39, 58,  1],
        [25, 17, 27, 10,  0, 21,  1, 54]])
targets:
torch.Size([4, 8])
tensor([[43, 58,  5, 57,  1, 46, 43, 39],
        [53, 56,  1, 58, 46, 39, 58,  1],
        [58,  1, 58, 46, 39, 58,  1, 46],
        [17, 27, 10,  0, 21,  1, 54, 39]])
----
when input is [24] the target is: 43
when input is [24, 43] the target is: 58
when input is [24, 43, 58] the target is: 5
when input is [24, 43, 58, 5] the target is: 57
when input is [24, 43, 58, 5, 57] the target is: 1
when input is [24, 43, 58, 5, 57, 1] the target is: 46
when input is [24, 43, 58, 5, 57, 1, 46] the target is: 43
when input is [24, 43, 58, 5, 57, 1, 46, 43] the target is: 39
when input is [44] the target is: 5

In [38]:
import torch.nn as nn
from torch.nn import functional as F
torch.manual_seed(1337)

class BigramLanguageModel(nn.Module):

    def __init__(self, vocab_size):
        super().__init__()
        # we have 64 individual tokens we want to embed (first arg), and the embedding dimension is going to be at least as large as 
        # the entire character space.
        self.token_embedding_table = nn.Embedding(vocab_size, vocab_size) # output is (batch, block_size, embed dim)

    def forward(self, idx, targets):
        logits = self.token_embedding_table(idx)
        # FROM PYTORCH DOCS:
        # INPUT:
        # Shape can be one of the following:
        # A 1D shape of size (C) where C is the number of classes.
        # A 2D shape of size (N, C) where N is the batch size and C is the number of classes.
        # A more complex shape of size (N, C, d1, d2, ..., dK) where N is the batch size, C is the number of classes, 
        # and d1, d2, ..., dK are additional dimensions. This happens when you have a K-dimensional loss, 
        # and K is greater than or equal to 1.
        
        # TARGET:
        # If the target contains class indices, its shape can be:
        # An empty shape (), representing a single value.
        # A 1D shape of size (N) where N is the batch size.
        # A more complex shape of size (N, d1, d2, ..., dK) where N is the batch size and d1, d2, ..., dK are additional dimensions, similar to the input case where you have a K-dimensional loss, and K is greater than or equal to 1.
        B, T, C = logits.shape
        logits = logits.view(B*T, C) # combine the batch * and blocks all into one,   
        targets = targets.view(B*T) #squash down to a single column
        loss = F.cross_entropy(logits, targets)
        
        return logits, loss

m = BigramLanguageModel(vocab_size)
logits, loss = m(xb, yb)
print(logits.shape, out.shape)

torch.Size([32, 65]) torch.Size([4, 8, 65])
